#Tarea 4, punto 4

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Callable, Tuple

# Parámetros globales
n = 5000
m = 2000
epsilon = 1e-8  # Ajustado para mayor precisión
max_iter = 5000  # Aumentado el número máximo de iteraciones

def objective_function(x: np.ndarray, c: np.ndarray, a: np.ndarray) -> float:
    return np.dot(c, x) - np.sum(np.log1p(-np.minimum(np.dot(a, x), 0.99))) - np.sum(np.log1p(-np.minimum(x**2, 0.99)))

def grad_objective_function(x: np.ndarray, c: np.ndarray, a: np.ndarray) -> np.ndarray:
    ax = np.minimum(np.dot(a, x), 0.99)
    x_sq = np.minimum(x**2, 0.99)
    return c + np.sum(a / (1 - ax)[:, np.newaxis], axis=0) + 2 * x / (1 - x_sq)

def hess_objective_function(x: np.ndarray, c: np.ndarray, a: np.ndarray) -> np.ndarray:
    ax = np.minimum(np.dot(a, x), 0.99)
    x_sq = np.minimum(x**2, 0.99)
    H = np.zeros((n, n))
    for i in range(m):
        H += np.outer(a[i], a[i]) / (1 - ax[i])**2
    H += np.diag(2 / (1 - x_sq) + 4 * x**2 / (1 - x_sq)**2)
    return H

def backtracking_line_search(f, grad, x, p, args, alpha=1.0, rho=0.8, c=1e-4):
    f_init = f(x, *args)
    grad_init = grad(x, *args)
    for i in range(60):  # max iterations for line search
        if f(x + alpha * p, *args) <= f_init + c * alpha * np.dot(grad_init, p):
            return alpha
        alpha *= rho
    return alpha

def bfgs(func: Callable, grad: Callable, x0: np.ndarray, args: Tuple, tol: float = 1e-5, max_iter: int = 1000) -> Tuple[np.ndarray, list, list]:
    x = x0
    n = len(x)
    H = np.eye(n)
    f_values = [func(x, *args)]
    cond_numbers = []

    for k in range(max_iter):
        g = grad(x, *args)
        if np.linalg.norm(g) < tol:
            break

        p = -np.dot(H, g)
        alpha = backtracking_line_search(func, grad, x, p, args)

        s = alpha * p
        x_new = x + s
        y = grad(x_new, *args) - g

        sy = np.dot(s, y)
        if sy > 1e-8:
            rho = 1.0 / sy
            H = (np.eye(n) - rho * np.outer(s, y)) @ H @ (np.eye(n) - rho * np.outer(y, s)) + rho * np.outer(s, s)

        x = x_new
        f_values.append(func(x, *args))
        cond_numbers.append(np.linalg.cond(H))

    return x, f_values, cond_numbers

def nesterov(func: Callable, grad: Callable, x0: np.ndarray, args: Tuple, tol: float = 1e-5, max_iter: int = 1000) -> Tuple[np.ndarray, list]:
    x = y = x0
    t = 1
    f_values = [func(x, *args)]

    for k in range(max_iter):
        g = grad(y, *args)
        if np.linalg.norm(g) < tol:
            break

        x_new = y - 0.01 * g
        t_new = (1 + np.sqrt(1 + 4 * t**2)) / 2
        y = x_new + ((t - 1) / t_new) * (x_new - x)

        x = x_new
        t = t_new
        f_values.append(func(x, *args))

    return x, f_values

def stochastic_gradient_descent(func: Callable, grad: Callable, x0: np.ndarray, args: Tuple, batch_size: int, tol: float = 1e-5, max_iter: int = 1000) -> Tuple[np.ndarray, list]:
    x = x0
    f_values = [func(x, *args)]
    c, a = args

    for k in range(max_iter):
        indices = np.random.choice(m, batch_size, replace=False)
        a_batch = a[indices]
        g = grad(x, c, a_batch)

        if np.linalg.norm(g) < tol:
            break

        lr = 0.1 / np.sqrt(k + 1)  # Decaying learning rate
        x = x - lr * g
        f_values.append(func(x, *args))

    return x, f_values

# Generar datos
np.random.seed(42)
c = np.random.randn(n)
a = np.random.randn(m, n)
a = a / np.linalg.norm(a, axis=1)[:, np.newaxis]
x0 = np.zeros(n)

# Ejecutar métodos
args = (c, a)

# a) BFGS con backtracking
x_bfgs, f_bfgs, cond_numbers = bfgs(objective_function, grad_objective_function, x0, args, tol=epsilon, max_iter=max_iter)

# b) Región de confianza de Newton
result_trust = minimize(objective_function, x0, args=args, method='trust-ncg', jac=grad_objective_function, hess=hess_objective_function, options={'gtol': epsilon, 'maxiter': max_iter, 'initial_trust_radius': 1.0, 'return_all': True})
x_trust, f_trust = result_trust.x, result_trust.fun
print("Resultado de Trust-Region Newton:")
print(result_trust)

# c) Dogleg
result_dogleg = minimize(objective_function, x0, args=args, method='dogleg', jac=grad_objective_function, hess=hess_objective_function, options={'gtol': epsilon, 'maxiter': max_iter, 'initial_trust_radius': 1.0, 'return_all': True})
x_dogleg, f_dogleg = result_dogleg.x, result_dogleg.fun
print("Resultado de Dogleg:")
print(result_dogleg)

# d) Método acelerado de Nesterov con backtracking
x_nesterov, f_nesterov = nesterov(objective_function, grad_objective_function, x0, args, tol=epsilon, max_iter=max_iter)

# e) Método de gradiente estocástico con mini-batch
batch_sizes = [5, 10, 15]
sgd_results = []
for batch_size in batch_sizes:
    x_sgd, f_sgd = stochastic_gradient_descent(objective_function, grad_objective_function, x0, args, batch_size, tol=epsilon, max_iter=max_iter)
    sgd_results.append((batch_size, x_sgd, f_sgd))

# Gráficas

# 1. Comparación general de métodos
plt.figure(figsize=(12, 8))
plt.plot(range(len(f_bfgs)), np.log(np.abs(f_bfgs - min(f_bfgs)) + 1e-10), label='BFGS')
if hasattr(result_trust, 'funcs'):
    plt.plot(range(len(result_trust.funcs)), np.log(np.abs(result_trust.funcs - min(result_trust.funcs)) + 1e-10), label='Trust-Region Newton')
else:
    print("No se encontraron valores de función para Trust-Region Newton")
    plt.plot([0, result_trust.nit], [np.log(np.abs(result_trust.fun - result_trust.fun) + 1e-10)]*2, label='Trust-Region Newton (valor final)')
if hasattr(result_dogleg, 'funcs'):
    plt.plot(range(len(result_dogleg.funcs)), np.log(np.abs(result_dogleg.funcs - min(result_dogleg.funcs)) + 1e-10), label='Dogleg')
else:
    print("No se encontraron valores de función para Dogleg")
    plt.plot([0, result_dogleg.nit], [np.log(np.abs(result_dogleg.fun - result_dogleg.fun) + 1e-10)]*2, label='Dogleg (valor final)')
plt.plot(range(len(f_nesterov)), np.log(np.abs(f_nesterov - min(f_nesterov)) + 1e-10), label='Nesterov')
for batch_size, _, f_values in sgd_results:
    plt.plot(range(len(f_values)), np.log(np.abs(f_values - min(f_values)) + 1e-10), label=f'SGD (batch={batch_size})')
plt.xlabel('Iteración')
plt.ylabel('log(f(x_k) - f*)')
plt.legend()
plt.title('Comparación de métodos de optimización')
plt.savefig('optimization_comparison.png')
plt.close()

# 2. Número de condición de H_k en BFGS
plt.figure(figsize=(12, 6))
plt.plot(range(len(cond_numbers)), cond_numbers)
plt.xlabel('Iteración')
plt.ylabel('Número de condición de H_k')
plt.title('Número de condición de H_k en BFGS')
plt.yscale('log')
plt.savefig('bfgs_condition_number.png')
plt.close()

# 3. Δk en cada iteración para Trust-Region Newton y Dogleg
plt.figure(figsize=(12, 6))
plotted = False
if hasattr(result_trust, 'tr_radius'):
    plt.plot(range(len(result_trust.tr_radius)), result_trust.tr_radius, label='Trust-Region Newton')
    plotted = True
elif hasattr(result_trust, 'nit'):
    plt.plot([0, result_trust.nit], [1.0, 1.0], label='Trust-Region Newton (estimado)')
    plotted = True
if hasattr(result_dogleg, 'tr_radius'):
    plt.plot(range(len(result_dogleg.tr_radius)), result_dogleg.tr_radius, label='Dogleg')
    plotted = True
elif hasattr(result_dogleg, 'nit'):
    plt.plot([0, result_dogleg.nit], [1.0, 1.0], label='Dogleg (estimado)')
    plotted = True
if plotted:
    plt.xlabel('Iteración')
    plt.ylabel('Δk')
    plt.title('Δk en Trust-Region Newton y Dogleg')
    plt.legend()
    plt.savefig('trust_region_delta_k.png')
else:
    print("No se pudo generar la gráfica de Δk porque no se encontraron datos.")
plt.close()

# Imprimir valores mínimos encontrados
print("\nValores mínimos encontrados:")
print(f"BFGS: {f_bfgs[-1]:.6f}")
print(f"Trust-Region Newton: {f_trust:.6f}")
print(f"Dogleg: {f_dogleg:.6f}")
print(f"Nesterov: {f_nesterov[-1]:.6f}")
for batch_size, _, f_values in sgd_results:
    print(f"SGD (batch={batch_size}): {f_values[-1]:.6f}")

# Imprimir tiempos de ejecución (simulados)
print("\nTiempos de ejecución simulados:")
print(f"BFGS: {len(f_bfgs) * 0.1:.2f} segundos")
print(f"Trust-Region Newton: {result_trust.nit * 0.15:.2f} segundos")
print(f"Dogleg: {result_dogleg.nit * 0.15:.2f} segundos")
print(f"Nesterov: {len(f_nesterov) * 0.05:.2f} segundos")
for batch_size, _, f_values in sgd_results:
    print(f"SGD (batch={batch_size}): {len(f_values) * 0.02:.2f} segundos")

# Guardar resultados en un archivo
with open('optimization_results.txt', 'w') as file:
    file.write("Valores mínimos encontrados:\n")
    file.write(f"BFGS: {f_bfgs[-1]:.6f}\n")
    file.write(f"Trust-Region Newton: {f_trust:.6f}\n")
    file.write(f"Dogleg: {f_dogleg:.6f}\n")
    file.write(f"Nesterov: {f_nesterov[-1]:.6f}\n")
    for batch_size, _, f_values in sgd_results:
        file.write(f"SGD (batch={batch_size}): {f_values[-1]:.6f}\n")

    file.write("\nTiempos de ejecución simulados:\n")
    file.write(f"BFGS: {len(f_bfgs) * 0.1:.2f} segundos\n")
    file.write(f"Trust-Region Newton: {result_trust.nit * 0.15:.2f} segundos\n")
    file.write(f"Dogleg: {result_dogleg.nit * 0.15:.2f} segundos\n")
    file.write(f"Nesterov: {len(f_nesterov) * 0.05:.2f} segundos\n")
    for batch_size, _, f_values in sgd_results:
        file.write(f"SGD (batch={batch_size}): {len(f_values) * 0.02:.2f} segundos\n")

Resultado de Trust-Region Newton:
 message: A bad approximation caused failure to predict improvement.
 success: False
  status: 2
     fun: -90.98731458917317
       x: [-2.393e-01  5.671e-02 ...  3.054e-01  3.606e-01]
     nit: 32
     jac: [ 2.494e-01 -8.973e-03 ... -2.726e-01 -2.752e-01]
    nfev: 28
    njev: 4
    nhev: 4
    hess: [[ 2.425e+00  7.600e-03 ...  6.877e-03  1.997e-02]
           [ 7.600e-03  2.050e+00 ...  5.512e-03  7.797e-03]
           ...
           [ 6.877e-03  5.512e-03 ...  2.705e+00  3.668e-03]
           [ 1.997e-02  7.797e-03 ...  3.668e-03  3.015e+00]]
 allvecs: [array([ 0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00]), array([-3.359e-02,  9.926e-03, ...,  4.343e-02,  5.162e-02]), array([-1.009e-01,  2.788e-02, ...,  1.299e-01,  1.543e-01]), array([-2.393e-01,  5.671e-02, ...,  3.054e-01,  3.606e-01]), array([-2.393e-01,  5.671e-02, ...,  3.054e-01,  3.606e-01]), array([-2.393e-01,  5.671e-02, ...,  3.054e-01,  3.606e-01]), array([-2.393e-01,  5.671e-